In [ ]:
# Required for importing modules from parent directory
import os
import sys

import soccer_twos

from src.config import N_GAMES
from src.utils import shape_rewards

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
sys.path.append(parent_dir)

In [ ]:
CHECKPOINT_DIR = "src\runs\out\checkpoints\checkpoints.zip"

In [ ]:
import random

import numpy as np


def evaluate_agent(agent, n_games=N_GAMES):
    env = soccer_twos.make(render=True, time_scale=1, quality_level=5, worker_id=3)

    for i in range(n_games):
        obs = env.reset()
        done = False
        scores = {}
        i = 0
        while not done:
            i += 1
            actions = {}
            for j in range(4):
                actions[j] = [0, 0, 0]
            actions[0], _ = agent.predict(np.array(obs[0]))

            next_obs, reward, done, info = env.step(actions)
            print(
                "[Agent 0] Reward: ",
                reward[0],
                " Score: ",
                shape_rewards(info, 0),
                "Action: ",
                actions[0],
            )
            done = done["__all__"]
            shape_rewards(info, 0)
            obs = next_obs

    env.close()

In [ ]:
from pathlib import Path

from stable_baselines3 import A2C

best_agent = A2C.load(Path(r"src\runs\out\checkpoints\bestmodel_a2c_5.zip"))
# Evaluate the trained agent
evaluate_agent(best_agent, n_games=5)

In [ ]:
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import load_results

df = load_results("out/logs/plain/")
df.head()

In [ ]:
import struct


def compress_floats(a: float, b: float) -> float:
    """Compress two 32-bit floats into one 64-bit float."""
    a_bits = struct.unpack("I", struct.pack("f", a))[0]
    b_bits = struct.unpack("I", struct.pack("f", b))[0]
    combined = (a_bits << 32) | b_bits
    return struct.unpack("d", struct.pack("Q", combined))[0]


def decompress_floats(c: float) -> tuple[float, float]:
    """Decompress one 64-bit float back into two 32-bit floats."""
    combined = struct.unpack("Q", struct.pack("d", c))[0]
    a_bits = combined >> 32
    b_bits = combined & 0xFFFFFFFF
    a = struct.unpack("f", struct.pack("I", a_bits))[0]
    b = struct.unpack("f", struct.pack("I", b_bits))[0]
    return a, b


# Example usage:
a, b = -0.244444, -0.9999
compressed = compress_floats(a, b)
a_recovered, b_recovered = decompress_floats(compressed)

print(f"Original: {a}, {b}")
print(f"Compressed: {compressed}")
print(f"Recovered: {a_recovered}, {b_recovered}")